In [118]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from nltk.corpus import stopwords
import string
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
import xgboost as xgb
import gc
import re, string
from sklearn.feature_extraction.text import TfidfVectorizer
import scipy

In [119]:
stop_words = set(stopwords.words('english'))

In [120]:
sample_submission = pd.read_csv("../data/sample_submission.csv")
train = pd.read_csv("../data/train.csv")
test = pd.read_csv("../data/test.csv")

In [121]:
train = train.fillna("unknown")
test = test.fillna("unknown")

In [122]:
sample_submission.head(1)

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.5,0.5,0.5,0.5,0.5,0.5


In [123]:
train.head(1)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\r\nWhy the edits made under my use...,0,0,0,0,0,0


In [124]:
test.head(1)

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...


In [125]:
def preprocess_text(sen):
    sentence = sen.lower()
    sentence = re.sub('\[.*?\]', '', sentence)
    sentence = re.sub('https?://\S+|www\.\S+', '', sentence)
    sentence = re.sub('<.*?>+', '', sentence)
    sentence = re.sub('[%s]' % re.escape(string.punctuation), '', sentence)
    sentence = re.sub('\n', '', sentence)
    sentence = re.sub('\w*\d\w*', '', sentence)
    # Удаление символов пунктуации и специальных символов
    sentence = re.sub('[^a-zA-Z0-9]', ' ', sen)
    # Заменяем любые пробелы на один обычный
    sentence = re.sub(r'\s+', ' ', sentence)
    # Удаляем стоп-слова
    sentence = " ".join([word for word in sentence.split() if word not in stop_words])
    return sentence

In [126]:
# filtered_train = train['comment_text'].apply(preprocess_text)
# filtered_test = test['comment_text'].apply(preprocess_text)

In [127]:
# def make_tf_idf(docs: pd.Series, stop_words=None):
#     return TfidfVectorizer(stop_words=stop_words).fit_transform(docs)

In [128]:
# tfidf_train = make_tf_idf(filtered_train)
# tfidf_test = make_tf_idf(filtered_test)

In [129]:
# print(tfidf_test)

In [130]:
# def preprocess_text(sen):
#     # Удаление символов пунктуации и специальных символов
#     sentence = re.sub('[^a-zA-Z0-9]', ' ', sen)
#     # Заменяем любые пробелы на один обычный
#     sentence = re.sub(r'\s+', ' ', sentence)
#     # Делаем всё в нижнем регистре
#     # Удаляем стоп-слова
#     sentence = " ".join([word for word in sentence.split() if word not in stop_words])
#     return sentence

In [131]:
filtered_train = train['comment_text'].apply(preprocess_text)
filtered_test = test['comment_text'].apply(preprocess_text)

In [132]:
filtered_train.head()

0    Explanation Why edits made username Hardcore M...
1    D aww He matches background colour I seemingly...
2    Hey man I really trying edit war It guy consta...
3    More I make real suggestions improvement I won...
4                You sir hero Any chance remember page
Name: comment_text, dtype: object

In [133]:
filtered_test.head()

0    Yo bitch Ja Rule succesful ever whats hating s...
1                          From RfC The title fine IMO
2                          Sources Zawe Ashton Lapland
3    If look back source information I updated corr...
4                          I anonymously edit articles
Name: comment_text, dtype: object

In [134]:
train_X, valid_X, train_Y, valid_Y = train_test_split(
    filtered_train,
    train[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']],
    test_size=0.2,
    random_state=20)

In [135]:
token_template = re.compile(f'([{string.punctuation}“”¨«»®´·º½¾¿¡§£₤‘’])')
def tokenizer_template(s):
    return token_template.sub(r' \1 ', s).split()

In [136]:
tf_idf_transform = TfidfVectorizer(ngram_range=(1, 2), tokenizer=tokenizer_template,
                                   min_df=3, max_df=0.9, strip_accents='unicode', use_idf=1,
                                   smooth_idf=1, sublinear_tf=1).fit(train['comment_text'])
'''comments_train = transform_com.transform(train['comment_text'])'''
tf_idf_train = tf_idf_transform.transform(train_X)
tf_idf_valid = tf_idf_transform.transform(valid_X)
tf_idf_test = tf_idf_transform.transform(test['comment_text'])

C:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\_param_validation.py:558: FutureWarning: Passing an int for a boolean parameter is deprecated in version 1.2 and won't be supported anymore in version 1.4.
  warnings.warn(
C:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [137]:
gc.collect()

8

In [138]:
train_X = pd.DataFrame(train_X)
valid_X = pd.DataFrame(valid_X)

In [139]:
data = [train_X, valid_X, test]

In [140]:
for element in data:
    element['total_length'] = element['comment_text'].apply(len)
    element['capitals'] = element['comment_text'].apply(lambda comment: sum(1 for c in comment if c.isupper()))
    element['num_exclamation_marks'] = element['comment_text'].apply(lambda comment: comment.count('!'))
    element['num_question_marks'] = element['comment_text'].apply(lambda comment: comment.count('?'))
    element['num_punctuation'] = element['comment_text'].apply(lambda comment: sum(comment.count(w) for w in '.,;:'))
    element['num_symbols'] = element['comment_text'].apply(lambda comment: sum(comment.count(w) for w in '*&$%'))
    element['num_words'] = element['comment_text'].apply(lambda comment: len(comment.split()))
    element['num_unique_words'] = element['comment_text'].apply(lambda comment: len(set(w for w in comment.split())))
    element['words_vs_unique'] = element['num_unique_words'] / element['num_words']
    element['num_smilies'] = element['comment_text'].apply(lambda comment: sum(comment.count(w) for w in (':-)', ':)', ';-)', ';)')))

col = ['total_length', 'capitals',
       'num_exclamation_marks', 'num_question_marks', 'num_punctuation',
       'num_symbols', 'num_words', 'num_unique_words', 'words_vs_unique',
       'num_smilies']

In [141]:
train_X = scipy.sparse.csr_matrix(train_X[col].values)
valid_X = scipy.sparse.csr_matrix(valid_X[col].values)
test = scipy.sparse.csr_matrix(test[col].values)

In [142]:
tf_idf_train = scipy.sparse.hstack([train_X.tocsr(), tf_idf_train.tocsr()])
tf_idf_valid = scipy.sparse.hstack([valid_X, tf_idf_valid])
tf_idf_test = scipy.sparse.hstack([test, tf_idf_test])

In [143]:
def runXGB(train_X, train_y, test_X, test_y=None, feature_names=None, seed_val=2017, num_rounds=500):
    param = {}
    param['objective'] = 'binary:logistic'
    param['eta'] = 0.1
    param['max_depth'] = 6
    param['silent'] = 1
    param['eval_metric'] = 'auc'
    param['min_child_weight'] = 1
    param['subsample'] = 0.7
    param['colsample_bytree'] = 0.7
    param['seed'] = seed_val
    num_rounds = num_rounds

    plst = list(param.items())
    xgtrain = xgb.DMatrix(train_X, label=train_y)

    if test_y is not None:
        xgtest = xgb.DMatrix(test_X, label=test_y)
        watchlist = [ (xgtrain,'train'), (xgtest, 'test') ]
        model = xgb.train(plst, xgtrain, num_rounds, watchlist, early_stopping_rounds=20)
    else:
        xgtest = xgb.DMatrix(test_X)
        model = xgb.train(plst, xgtrain, num_rounds)

    return model

In [144]:
col = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
preds = np.zeros((test.shape[0], len(col)))

In [145]:
for i, j in enumerate(col):
    print('fit '+j)
    model = runXGB(tf_idf_train, train_Y[j], tf_idf_valid, valid_Y[j])
    preds[:,i] = model.predict(xgb.DMatrix(tf_idf_test), ntree_limit = model.best_ntree_limit)

fit toxic
[16:16:19] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07593ffd91cd9da33-1\xgboost\xgboost-ci-windows\src\learner.cc:767: 
Parameters: { "silent" } are not used.



C:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:617: FutureWarning: Pass `evals` as keyword args.
  warnings.warn(msg, FutureWarning)


[0]	train-auc:0.65038	test-auc:0.64452
[1]	train-auc:0.70559	test-auc:0.70203
[2]	train-auc:0.74818	test-auc:0.74729
[3]	train-auc:0.74813	test-auc:0.74721
[4]	train-auc:0.75404	test-auc:0.75451
[5]	train-auc:0.78605	test-auc:0.78950
[6]	train-auc:0.79379	test-auc:0.79687
[7]	train-auc:0.79653	test-auc:0.79815
[8]	train-auc:0.80121	test-auc:0.80336
[9]	train-auc:0.80754	test-auc:0.80910
[10]	train-auc:0.81447	test-auc:0.81367
[11]	train-auc:0.84527	test-auc:0.84143
[12]	train-auc:0.84581	test-auc:0.84177
[13]	train-auc:0.84999	test-auc:0.84563
[14]	train-auc:0.85790	test-auc:0.85328
[15]	train-auc:0.86087	test-auc:0.85555
[16]	train-auc:0.86386	test-auc:0.85870
[17]	train-auc:0.86969	test-auc:0.86711
[18]	train-auc:0.87284	test-auc:0.86971
[19]	train-auc:0.87601	test-auc:0.87260
[20]	train-auc:0.87757	test-auc:0.87330
[21]	train-auc:0.88237	test-auc:0.87828
[22]	train-auc:0.88777	test-auc:0.88399
[23]	train-auc:0.88946	test-auc:0.88535
[24]	train-auc:0.89212	test-auc:0.88842
[25]	train

C:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:122: UserWarning: ntree_limit is deprecated, use `iteration_range` or model slicing instead.
  warnings.warn(


fit severe_toxic
[16:19:59] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07593ffd91cd9da33-1\xgboost\xgboost-ci-windows\src\learner.cc:767: 
Parameters: { "silent" } are not used.

[0]	train-auc:0.77144	test-auc:0.76428
[1]	train-auc:0.88132	test-auc:0.87993
[2]	train-auc:0.89936	test-auc:0.89356
[3]	train-auc:0.90586	test-auc:0.89829
[4]	train-auc:0.90829	test-auc:0.90025
[5]	train-auc:0.91415	test-auc:0.90494
[6]	train-auc:0.91598	test-auc:0.90634
[7]	train-auc:0.91685	test-auc:0.90639
[8]	train-auc:0.92143	test-auc:0.91555
[9]	train-auc:0.92497	test-auc:0.91543
[10]	train-auc:0.92760	test-auc:0.91863
[11]	train-auc:0.93330	test-auc:0.92480
[12]	train-auc:0.93341	test-auc:0.92477
[13]	train-auc:0.93347	test-auc:0.92475
[14]	train-auc:0.93400	test-auc:0.92641
[15]	train-auc:0.94335	test-auc:0.94163
[16]	train-auc:0.94381	test-auc:0.94322
[17]	train-auc:0.94903	test-auc:0.94925
[18]	train-auc:0.94911	test-auc:0.94931
[19]	train-auc:0.94916	test-auc:0.949

In [146]:
gc.collect()

82

In [147]:
subm = pd.read_csv('../data/sample_submission.csv')
sample_submission_id = pd.DataFrame({'id': subm["id"]})
submission = pd.concat([sample_submission_id, pd.DataFrame(preds, columns = col)], axis=1)
submission.to_csv('submission_3.csv', index=False)